In [1]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

### McM scripting [tutorial](https://indico.cern.ch/event/894961/contributions/3774016/attachments/2005333/3349474/McM_Scripting_2020_03_18.pdf)
### Documentation: [restapi](https://cms-pdmv.cern.ch/mcm/restapi)

In [2]:
mcm = McM(dev=False)
tags = '20231101_jreicher_SUEP_WH'
results = mcm.get('requests', query=f'tags={tags}&status=new&approval=none')
len(results)

[2023-11-13 09:49:17,897][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt
[2023-11-13 09:49:17,949][ERROR] Verifying credential: HTTP response has a 3XX status code
[2023-11-13 09:49:17,952][ERROR] The provided credential is not valid
[2023-11-13 09:49:17,952][INFO] Authentication mechanism: sso


[2023-11-13 09:49:20,202][WARNING] Python 3.X version currently used has reached its end of life or it will reach it in the near future.
Please consider using a newer version. Python version: 3.9.6 (default, Dec 16 2021, 17:01:31) 
[GCC 10.3.0] 



44

In [6]:
results[0]['member_of_campaign']

'RunIISummer20UL16wmLHEGEN'

In [10]:
CLEAR=True
if CLEAR:
    for r in results:
        mcm.delete(object_type='requests', object_id=r['_id'])

In [11]:
for r in results:
    print(r['member_of_campaign'])
    #r['member_of_campaign'] = r['member_of_campaign'].replace('wmLHEGEN', 'GEN')
    #r['fragment'] = r['fragment'].replace('generateConcurrently = cms.untracked.bool(True),', 'generateConcurrently = cms.untracked.bool(False),')
    #r['time_event'] = [13]
    #r['size_event'] = [1252]
    #r['mcdb_id'] = 0
    #print(r['dataset_name'].split('_M')[1].split('_')[0], r['fragment'].split("GRIDPACK = \'")[1].split("\' %")[0])

RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GEN
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL16GENAPV
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL17GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN
RunIISummer20UL18GEN


In [8]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'], output['message'])

EXO-RunIISummer20UL16wmLHEGEN-04615 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04616 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04617 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04618 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04619 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04620 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04621 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04622 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04623 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04624 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGEN-04625 Illegal change of parameter member_of_campaign
EXO-RunIISummer20UL16wmLHEGENAPV-04368 Illegal change of parameter member_of_campaign
E

In [6]:
time = set()
for r in results:
    time.add(r['time_event'][0])
time

{8}